In [39]:
from SPARQLWrapper import SPARQLWrapper, JSON, RDF
from rdflib import Graph 
from pyvis.network import Network
import json

import networkx as nx
import csv

#endPoint = "http://192.168.1.12:7200/repositories/TFT"
endPoint = "http://DESKTOP-TTM16LQ:7200/repositories/TFT"

sparql = SPARQLWrapper(endPoint)

# USE CASE 1: Comparisson of three index based on predicate:

##########

En las siguintes consultas cambiar el nombre del índice PSIM2 por PredicationIndex

##########


## 1. Index based on basic metadata
## 2. Index based on basic metadata base + dbr + dbc
## 3. Index based on basic metadata base + dbr + dbc + spacy

In [40]:
# Queries for indexes:

# Query based on basic metadata such as title, abstract, keywords and authors:
queryIdx1 = """
    PREFIX : <http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst: <http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX psi: <http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX schema: <http://schema.org/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX skg: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    PREFIX pred: <http://www.ontotext.com/graphdb/similarity/psi/>
    
    SELECT distinct ?entity ?id ?title (GROUP_CONCAT(DISTINCT ?keywords; SEPARATOR=",") AS ?keys) ?description 
            (GROUP_CONCAT(DISTINCT ?author; SEPARATOR=",") AS ?authors) ?score WHERE {
        ?search a inst:PredicationIndex1 ; psi:searchEntity <%s>; 
        psi:searchPredicate <http://www.ontotext.com/graphdb/similarity/psi/any>;
            psi:entityResult ?result .
        ?result :value ?entity ; 
            :score ?score.
        ?entity rdf:type schema:Article ; schema:identifier ?id .
        OPTIONAL { ?entity skg:title ?title .}
        OPTIONAL { ?entity skg:abstract ?description . }
        OPTIONAL { ?entity skg:keyword ?keywords . }
        OPTIONAL { ?entity dct:creator ?author . }
    }group by ?entity ?id ?title ?description ?score
    """

queryIdx2 = """
    PREFIX : <http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst: <http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX psi: <http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX schema: <http://schema.org/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX skg: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    PREFIX pred: <http://www.ontotext.com/graphdb/similarity/psi/>
    
    SELECT distinct ?entity ?id ?title ?description 
    (GROUP_CONCAT(DISTINCT ?dbr1; SEPARATOR=",") AS ?dbr) 
    (GROUP_CONCAT(DISTINCT ?dbc1; SEPARATOR=",") AS ?dbc) 
    (GROUP_CONCAT(DISTINCT ?keywords; SEPARATOR=",") AS ?keys) 
    (GROUP_CONCAT(DISTINCT ?author; SEPARATOR=",") AS ?authors) ?score
WHERE {
        ?search a inst:PredicationIndex2; psi:searchEntity <%s>; psi:searchPredicate <http://www.ontotext.com/graphdb/similarity/psi/any>;
            :searchParameters ""; psi:entityResult ?result .
        ?result :value ?entity ;
                :score ?score.
        ?entity rdf:type schema:Article ; schema:identifier ?id .
        OPTIONAL { ?entity skg:title ?title .}
        OPTIONAL { ?entity skg:abstract ?description . }
        OPTIONAL { ?entity skg:keyword ?keywords . }
        OPTIONAL { ?entity dct:creator ?author . }
        OPTIONAL { ?entity schema:mentions/rdfs:label ?dbr1 . }
        OPTIONAL { ?entity dct:subject/rdfs:label ?dbc1 . }
    }group by ?entity ?id ?title ?description ?score
    """

queryIdx3 = """
    PREFIX : <http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst: <http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX psi: <http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX schema: <http://schema.org/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX skg: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    PREFIX pred: <http://www.ontotext.com/graphdb/similarity/psi/>
    
    SELECT distinct ?id ?entity ?score ?title ?description 
(GROUP_CONCAT(DISTINCT ?spacySu; SEPARATOR=",") AS ?spacySujeto)
(GROUP_CONCAT(DISTINCT ?spacyPre; SEPARATOR=",") AS ?spacyPredicado)
(GROUP_CONCAT(DISTINCT ?spacyOb; SEPARATOR=",") AS ?spacyObjeto)
(GROUP_CONCAT(DISTINCT ?dbr1; SEPARATOR=",") AS ?dbr) 
(GROUP_CONCAT(DISTINCT ?dbc1; SEPARATOR=",") AS ?dbc) 
(GROUP_CONCAT(DISTINCT ?keywords; SEPARATOR=",") AS ?keys) 
(GROUP_CONCAT(DISTINCT ?author; SEPARATOR=",") AS ?authors) 
WHERE {
        ?search a inst:PredicationIndex3; psi:searchEntity <%s>; psi:searchPredicate <http://www.ontotext.com/graphdb/similarity/psi/any>;
            :searchParameters ""; psi:entityResult ?result .
        ?result :value ?entity ; :score ?score . 
        ?entity rdf:type schema:Article ; schema:identifier ?id .
        OPTIONAL { ?entity skg:title ?title .}
        OPTIONAL { ?entity skg:abstract ?description . }
        OPTIONAL { ?entity skg:keyword ?keywords . }
        OPTIONAL { ?entity dct:creator ?author . }
        OPTIONAL { ?entity schema:mentions/rdfs:label ?dbr1 . }
        OPTIONAL { ?entity dct:subject/rdfs:label ?dbc1 . }
        OPTIONAL {?entity schema:hasPart/schema:about/schema:subjectOf/schema:name ?spacyP .}
        OPTIONAL {?entity schema:hasPart/schema:about/schema:name ?spacyS .}
    }group by ?id ?entity ?title ?description ?score
    """

In [41]:
# To query:
def getData(idPaper, Q):
    resultList = []
    sparql.setQuery(Q%(idPaper))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    description = ''
    for result in results["results"]["bindings"]:
        if 'description' in result:
            description = result['description']['value']
        resultList.append([result['entity']['value'], result['id']['value'], result['title']['value'], description, result['authors']['value'], result['keys']['value'], result['score']['value'],])
        description = ' '
        #no permite sacar la variable "description"
    return resultList

## To run 3 index:

In [ ]:
#paperList = ["https://w3id.org/scholarlydata/inproceedings/www2011/demo/yago2-exploring-and-querying-world-knowledge-in-ti",
             #"https://w3id.org/scholarlydata/inproceedings/eswc2012/paper/research/73"]
paperList = ["https://w3id.org/scholarlydata/inproceedings/www2011/demo/yago2-exploring-and-querying-world-knowledge-in-ti"]

for uriPaper in paperList:

    print('\n\n ', 60*'-', 'Índex 1')
    resultsIdx1 = getData(uriPaper, queryIdx1)
    print(resultsIdx1[1:4])

    print('\n\n ', 60*'-', 'Índex 2')
    resultsIdx2 = getData(uriPaper, queryIdx2)
    print(resultsIdx2[1:4])

    print('\n\n ', 60*'-', 'Índex 3')
    resultsIdx3 = getData(uriPaper, queryIdx3)
    print(resultsIdx3[1:4])

# Index based on text:

# USE CASE 2: Search based on text index

## Query the index

In [ ]:
def getResults(keyword):
    SimilarityKeywords = """
    PREFIX :<http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst:<http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX pubo: <http://ontology.ontotext.com/publishing#>
    PREFIX schema: <http://schema.org/>
    PREFIX scholarly: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>

    SELECT ?documentID ?documentSec ?title ?abstract (GROUP_CONCAT(?keyword; SEPARATOR='; ') AS ?keywords) ?score {
    ?search a inst:SSTAK ;
        :searchTerm "%s";
        :searchParameters "";
        :documentResult ?result .
    ?result :value ?documentID ;
        :score ?score.
    ?documentID schema:identifier ?documentSec 
    OPTIONAL {
            ?documentID scholarly:title ?title .
    }
    OPTIONAL {
            ?documentID scholarly:title ?title .
    }
    OPTIONAL {
            ?documentID scholarly:abstract ?abstract .
    }
    OPTIONAL {
            ?documentID scholarly:keyword ?keyword .
    }
    } GROUP BY ?documentID ?documentSec ?title ?abstract  ?score ORDER BY DESC(?score)
    """
    resultList = []
    sparql.setQuery(SimilarityKeywords%(keyword))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        resultList.append([result['documentID']['value'], result['documentSec']['value'], result['title']['value'],
                           result['abstract']['value'], result['keywords']['value'], result['score']['value']])
    return resultList
        

In [ ]:
#resultados despues de ejecutar la funcion
results = getResults("Knowledge Representation")
results[0:2]

## Computing simmilarity:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#tsvfile = pd.read_csv(open("papersT.tsv", "r"), delimiter="\t", header=None)
tsvfile = pd.read_csv(open("papersD.tsv", "r"), delimiter="\t", header=None) # cambiar

tsvfile = tsvfile.values.tolist()
tsvfile[0:5]

In [ ]:
def verify(e):
    # To select triplets where ids are retrieved from GraphDB
    if e in ids:
        return True
    else:
        return False
    
def createSubGraph():
    # To create the subgraph based on ids of papers:
    subset = [[line[0], line[2]] for line in tsvfile if verify(int(line[0])) or verify(int(line[2]))]; print(subset[0:20])
    for line in subset:
        x, y = line
        g.add_edge(x, y)
    #nx.draw_circular(g, with_labels = True)
    #plt.savefig("filename1.png")
    #plt.clf()

def calculateSimm(min=0.2):
    # To compute the simmilarity matrix between selected entities.
    simMatrix = nx.simrank_similarity(g)
    simList = []
    for d in simMatrix:
        for dd in simMatrix[d]:
            if simMatrix[d][dd] >= min and simMatrix[d][dd] < 1:
                simList.append([d, dd, simMatrix[d][dd]])
    return simList

In [ ]:
# to run:
results = getResults("Semantic Web")

ids = [int(i[1]) for i in results]
print(ids)
g = nx.Graph()
createSubGraph()
print(g)
sim = calculateSimm()
print(sim[0:2])

In [ ]:
# filtra solo papers

print(len(sim)) # 8622

simSel = [[line[0], line[1], line[2]] for line in sim if verify(line[0]) or verify(line[1])]; len(simSel)

## Getting metadata from KG

In [ ]:
def getMetadata(idSec):
    searchMetada = """
    PREFIX schema: <http://schema.org/>
    PREFIX skg: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    select distinct ?entity ?title ?abstract 
    (GROUP_CONCAT(DISTINCT ?keywords; SEPARATOR=",") AS ?keys)
    where{  
        ?entity <http://schema.org/identifier> "%s".
        ?entity skg:title ?title.
        ?entity skg:abstract ?abstract .
        ?entity skg:keyword ?keywords .   
    }GROUP BY ?entity ?title ?abstract
    """
    resultList = []
    sparql.setQuery(searchMetada%(idSec))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        resultList.append([idSec, result['entity']['value'], result['title']['value'], result['abstract']['value'], result['keys']['value']])
    return resultList[0]

results = getMetadata(328)
results

In [ ]:
# Obtener los ids de los papers que se encuentran en los nodos de la matriz de similitud (obtener valores únicos)
simSelDF = pd.DataFrame(simSel, columns=['id1', 'id2', 'score'])
idsSim = list(set(simSelDF['id1'].tolist() + simSelDF['id2'].tolist())); idsSim


In [ ]:
#obtener los metadatos de todos los papers que salieron en la matriz de similitud:
metadataSim = pd.DataFrame()
for i in idsSim:
    m = getMetadata(i)
    metadataSim = metadataSim.append({'id':m[0], 'uri':m[1], 'title':m[2], 'abstract':m[3]}, ignore_index=True)
        
metadataSim.head() # Metadata de los papers de la matriz de similitud

In [ ]:
#add metadata a la matriz de similitud y convertir a lista:
simSelMeta = pd.merge(pd.DataFrame(simSel, columns=['id1', 'id2', 'score']), metadataSim, left_on="id1", right_on="id")
print(simSelMeta,"1")
simSelMeta.head()
simSelMeta = pd.merge(simSelMeta, metadataSim, left_on="id2", right_on="id")
print(simSelMeta,"2")
simSelMeta.head()
#simSelMeta = pd.merge(pd.DataFrame(simSel, columns=['id', 'id2', 'score']), metadataSim)
print(simSelMeta.head())
simSelMeta = simSelMeta.values.tolist()

## Visualization

Source:

https://towardsdatascience.com/python-interactive-network-visualization-using-networkx-plotly-and-dash-e44749161ed7

https://pyvis.readthedocs.io/en/latest/

In [ ]:
net = Network('1000px','1500px')
listUri = []
unoSolo = set()
for lines in simSelMeta:
    x = lines[6].replace('https://w3id.org/scholarlydata/', '')
    y = lines[10].replace('https://w3id.org/scholarlydata/', '')
    score = lines[2]
    # etiquetas:
    titlex = str(lines[5])
    print(titlex)
    titley = str(lines[9])

    title_x = '<a href='+ str(lines[4]) + ' target="_blank" >' + 'Paper Link' + '</a> <br> <b> Title: </b>'+ titlex
    title_y = '<a href='+ str(lines[8]) + ' target="_blank" >' + 'Paper Link' + '</a> <br> <b> Title: </b>'+ titley
    net.add_node(x, title=title_x, label=x)
    net.add_node(y, title=title_y, label=y) 
    net.add_edge(x, y, width=score*5, label = round(score, 2))
net.show_buttons(filter_=['physics'])

net.show("dot.html")

## To run a complete case

In [4]:
# Search:
results = getResults("Machine Learning")

#Create graph:
ids = [int(i[1]) for i in results]
g = nx.Graph()
createSubGraph()
sim = calculateSimm()

# filtra solo papers
simSel = [[line[0], line[1], line[2]] for line in sim if verify(line[0]) or verify(line[1])]; len(simSel)

# Obtener los ids de los papers que se encuentran en los nodos de la matriz de similitud (obtener valores únicos)
simSelDF = pd.DataFrame(simSel, columns=['id1', 'id2', 'score'])
idsSim = list(set(simSelDF['id1'].tolist() + simSelDF['id2'].tolist())); idsSim

#obtener los metadatos de todos los papers que salieron en la matriz de similitud:
metadataSim = pd.DataFrame()
for i in idsSim:
    m = getMetadata(i)
    metadataSim = metadataSim.append({'id':m[0], 'uri':m[1], 'title':m[2], 'abstract':m[3]}, ignore_index=True)
        
metadataSim.head() # Metadata de los papers de la matriz de similitud


#add metadata a la matriz de similitud y convertir a lista:
simSelMeta = pd.merge(pd.DataFrame(simSel, columns=['id1', 'id2', 'score']), metadataSim, left_on="id1", right_on="id")

simSelMeta = pd.merge(simSelMeta, metadataSim, left_on="id2", right_on="id")

#simSelMeta = pd.merge(pd.DataFrame(simSel, columns=['id', 'id2', 'score']), metadataSim)
simSelMeta = simSelMeta.values.tolist()

net = Network('1000px','1500px')
for lines in simSelMeta:
    x = lines[6].replace('https://w3id.org/scholarlydata/', '')
    y = lines[10].replace('https://w3id.org/scholarlydata/', '')
    score = lines[2]
    # etiquetas:
    title_x = '<a href='+ str(lines[4]) + ' target="_blank" >' + 'Paper Link' + '</a> <br> <b> Title: </b>'+str(lines[5])
    title_y = '<a href='+ str(lines[8]) + ' target="_blank" >' + 'Paper Link' + '</a> <br> <b> Title: </b>'+str(lines[9])
    net.add_node(x, title=title_x, label=x)
    net.add_node(y, title=title_y, label=y) 
    net.add_edge(x, y, width=score*5, label = round(score, 2))
net.show_buttons(filter_=['physics'])

net.show("case2_1.html")

NameError: name 'getResults' is not defined

# Index based on predictions:

## USE CASE 1: Similarity on predictions


# Validacion 

In [42]:
#https://goodboychan.github.io/python/datacamp/natural_language_processing/2020/07/17/04-TF-IDF-and-similarity-scores.html
import pandas as pd
import numpy as np
import spacy 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#def valoracionManual(resumen1, resumen2, titulo1, titulo2, palabrasClave1, palabraClave2):
def scoreSPACY(resumen1, resumen2):
    #nlp = spacy.load("en_core_web_sm")
    nlp = spacy.load("en_core_web_md")
    print(resumen1,"1111111")
    print(resumen2,"2222222")
    tokens1 = nlp(resumen1)
    tokens2 = nlp(resumen2)
    oraciones1 = []
    oraciones2 = []
    for sent in tokens1.sents:
        ora = sent.text.strip()
        oraciones1.append(ora)

    for sent in tokens2.sents:
        ora = sent.text.strip()
        oraciones2.append(ora)
    if(resumen2==' '):
        valorSim = 0
    else:
        docus = (oraciones1, oraciones2)
        # Initialize an instance of tf-idf Vectorizer
        tfidf_vectorizer = TfidfVectorizer()
        # Generate the tf-idf vectors for the corpus
        tfidf_matrix = tfidf_vectorizer.fit_transform(docus[0], docus[1])
        # compute and print the cosine similarity matrix
        cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
        valorSim = cosine_sim[0][0]    
    #print("similaridad de coseno -->", cosine_sim)
    return valorSim
    

In [43]:
def exPapers():
    extractPapers = """
    prefix scholarly: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    select distinct ?s { 
        ?s a <http://schema.org/Article> ;
        scholarly:abstract [];
        scholarly:title [];
    }
    """
    resultList = []
    sparql.setQuery(extractPapers)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        resultList.append(result['s']['value'])
    return resultList

## Seleccion aleatoria de papers y guardado en archivo Excel "VALIDACION MANUAL"

In [44]:
import random
import openpyxl 
import pandas as pd
import numpy as np

resultListIndex1 = []
resultListIndex2 = []
resultListIndex3 = []
baseId1 = []
baseId2 = []
baseId3 = []

contador1 = 0
contador2 = 0
contador3 = 0

papersBase1 =  pd.DataFrame()
papersReco1 =  pd.DataFrame()
papersBase2 =  pd.DataFrame()
papersReco2 =  pd.DataFrame()
papersBase3 =  pd.DataFrame()
papersReco3 =  pd.DataFrame()
index1DF = pd.DataFrame()
index2DF = pd.DataFrame()
index3DF = pd.DataFrame()
varVacia = ''
#se trae todos los papers en la variable result 
result = exPapers()
resultsList = []
#se guarda en una lista resultsList, los papers
for i in result:
    resultsList.append(i)    
#se guarda en una lista los papers generados aleatoriamente
num = 20
aleatorios = np.random.choice(resultsList, num, False)
print(aleatorios, "papers Aleatorios")
#De los papers generados aletoriamente, se los recorre para sacar los similares mediante la utilizacion del indice 1
for uriPaper1 in aleatorios:
    print('\n\n ', 60*'-', 'Índex 1')
    resultsIdx1 = getData(uriPaper1, queryIdx1)
    resultListIndex1 = resultsIdx1[1:4]
    print(resultListIndex1, "indice 1")
    #Metadatos del paper aleatorio
    baseId1 = resultsIdx1[0]
    index1 = "1 (basic metadata)"
    papersBase1 = papersBase1.append({'index':index1, 'uri bas':baseId1[0], 'title bas':baseId1[2], 'abstract bas':baseId1[3], 'keywords bas':baseId1[5]}, ignore_index=True)
    #Metadatos del paper recomendado
    for result1 in resultListIndex1:       
        if (contador1 == 3):
            contador1 = 0
        contador1 = contador1 + 1
        papersReco1 = papersReco1.append({'uri bas':baseId1[0], 'uri sim':result1[0], 'title sim':result1[2], 'abstract sim':result1[3], 'keywords sim':result1[5], 'score GraphDB':result1[6], 'Rank GraphDB':contador1, 'rating manually title':varVacia,'rating manually abstract':varVacia, 'rating manually keywords':varVacia,'average ratings':varVacia, 'Rank manually':varVacia, 'error':varVacia}, ignore_index=True)
        index1DF = papersBase1.merge(papersReco1, left_on = "uri bas", right_on = "uri bas")
    contador1 = 0   
#De los papers generados aletoriamente, se los recorre para sacar los similares mediante la utilizacion del indice 2       
for uriPaper2 in aleatorios:
    print('\n\n ', 60*'-', 'Índex 2')
    resultsIdx2 = getData(uriPaper2, queryIdx2)
    resultListIndex2 = resultsIdx2[1:4]
    print(resultListIndex2, "indice 2")
    #Metadatos del paper aleatorio
    baseId2 = resultsIdx2[0]
    index2 = "2 (basic metadata + dbpedia)"
    papersBase2 = papersBase2.append({'index':index2, 'uri bas':baseId2[0], 'title bas':baseId2[2], 'abstract bas':baseId2[3], 'keywords bas':baseId2[5]}, ignore_index=True)
    #Metadatos del paper recomendado
    for result2 in resultListIndex2:        
        if (contador2 == 3):
            contador2 = 0
        contador2 = contador2 + 1
        papersReco2 = papersReco2.append({'uri bas':baseId2[0], 'uri sim':result2[0], 'title sim':result2[2], 'abstract sim':result2[3], 'keywords sim':result2[5], 'score GraphDB':result2[6], 'Rank GraphDB':contador2,'rating manually title':varVacia,'rating manually abstract':varVacia, 'rating manually keywords':varVacia,'average ratings':varVacia, 'Rank manually':varVacia, 'error':varVacia}, ignore_index=True)
        index2DF = papersBase2.merge(papersReco2, left_on = "uri bas", right_on = "uri bas")
    contador2 = 0
#De los papers generados aletoriamente, se los recorre para sacar los similares mediante la utilizacion del indice 2       
for uriPaper3 in aleatorios:
    print('\n\n ', 60*'-', 'Índex 3')
    resultsIdx3 = getData(uriPaper3, queryIdx3)
    resultListIndex3 = resultsIdx3[1:4]
    print(resultListIndex3,"indice 3")
    #Metadatos del paper aleatorio
    baseId3 = resultsIdx3[0]
    index3 = "3 (basic metadata + dbpedia + spacy)"
    papersBase3 = papersBase3.append({'index':index3, 'uri bas':baseId3[0], 'title bas':baseId3[2], 'abstract bas':baseId3[3], 'keywords bas':baseId3[5]}, ignore_index=True)
    #Metadatos del paper recomendado
    for result3 in resultListIndex3:        
        if (contador3 == 3):
            contador3 = 0
        contador3 = contador3 + 1
        papersReco3 = papersReco3.append({'uri bas':baseId3[0], 'uri sim':result3[0], 'title sim':result3[2], 'abstract sim':result3[3], 'keywords sim':result3[5], 'score GraphDB':result3[6], 'Rank GraphDB':contador3,'rating manually title':varVacia,'rating manually abstract':varVacia, 'rating manually keywords':varVacia,'average ratings':varVacia, 'Rank manually':varVacia, 'error':varVacia}, ignore_index=True)
        index3DF = papersBase3.merge(papersReco3, left_on = "uri bas", right_on = "uri bas")
    contador3 = 0
#union de los 3 dataframes mediante .concat
pdList = [index1DF, index2DF, index3DF]
vmDF = pd.concat(pdList)
#creacion del archivo excel "validacion_manual"
vmDF.to_excel('validacion_manual.xlsx')
#simIndex1.to_excel('validacion_manual.xlsx')

['https://w3id.org/scholarlydata/inproceedings/iswc2013/poster-demo-proceedings/paper-08'
 'https://w3id.org/scholarlydata/inproceedings/lrec2008/papers/322'
 'https://w3id.org/scholarlydata/inproceedings/lrec2008/papers/313'
 'https://w3id.org/scholarlydata/inproceedings/iswc2012/proceedings-2/paper-26'
 'https://w3id.org/scholarlydata/inproceedings/lrec2008/papers/851'
 'https://w3id.org/scholarlydata/inproceedings/eswc2008/paper/90'
 'https://w3id.org/scholarlydata/inproceedings/www2011/phd/sentence-level-contextual-opinion-retrieval'
 'https://w3id.org/scholarlydata/inproceedings/eswc2016/paper/research/219'
 'https://w3id.org/scholarlydata/inproceedings/www2011/poster/leveraging-auxiliary-text-terms-for-automatic-imag'
 'https://w3id.org/scholarlydata/inproceedings/lrec2008/papers/648'
 'https://w3id.org/scholarlydata/inproceedings/lrec2008/papers/324'
 'https://w3id.org/scholarlydata/inproceedings/lrec2008/papers/786'
 'https://w3id.org/scholarlydata/inproceedings/www2012/paper/5

[['https://w3id.org/scholarlydata/inproceedings/dh2010/abstracts/poster/ab-797', '134', 'Generation of Emotional Dance Motion for Virtual Dance Collaboration System', ' ', 'https://w3id.org/scholarlydata/person/choi-woong,https://w3id.org/scholarlydata/person/hachimura-kozaburo,https://w3id.org/scholarlydata/person/tsuruta-seiya', '', '0.2779412268172163'], ['https://w3id.org/scholarlydata/inproceedings/iswc2010/paper/414', '2191', 'Mapping Master: a Flexible Approach for Mapping Spreadsheets to OWL', 'We describe a mapping language for converting data contained in spreadsheets into the Web Ontology Language (OWL). The developed language, called M2, overcomes shortcomings with existing mapping techniques, including their restriction to well-formed spreadsheets reminiscent of a single relational database table and verbose syntax for expressing mapping rules when transforming spreadsheet contents into OWL. The M2 language provides expressive, yet concise mechanisms to create both individ

[['https://w3id.org/scholarlydata/inproceedings/www2007/paper/main/759', '3715', 'Compare&Contrast: Using the Web to Discover Comparable Cases for News Stories', 'Comparing and contrasting is an important strategy people adopt to understand new situations and create solutions to for new problems. Similar events can provide hints for problem solving, as well as larger contexts for understanding the specific circumstances of an event. Lessons from past experience can be applied, insights can be gained about the new situation from familiar examples, and trends can be discovered among similar events. As the largest knowledge base for human beings, the Web provides both an opportunity and a challenge to discover comparable cases in order to facilitate situation analysis and problem solving. In this paper, we present Compare&Contrast, a system that uses the Web to discover comparable cases for news stories, documents about similar situations but involving distinct entities. The system analyz

[['https://w3id.org/scholarlydata/inproceedings/lrec2008/papers/574', '3055', 'F0 of Adolescent Speakers   First Results for the German Ph@ttSessionz Database', 'The first release of the German Ph@ttSessionz speech database contains read and spontaneous speech from 864 adolescent speakers and is the largest database of its kind for German. It was recorded via the WWW in over 40 public schools in all dialect regions of Germany. In this paper, we present a cross-sectional study of f0 measurements on this database. The study documents the profound changes in male voices at the age 13-15. Furthermore, it shows that on a perceptive mel-scale, there is little difference in the relative f0 variability for male and female speakers. A closer analysis reveals that f0 variability is dependent on the speech style and both the length and the type of the utterance. The study provides statistically reliable voice parameters of adolescent speakers for German. The results may contribute to making spoke

[['https://w3id.org/scholarlydata/inproceedings/iswc2010/paper/529', '2255', 'A Semantic Web Repository for Managing and Querying Aligned Knowledge', 'Ontology alignment is the task of matching concepts and terminology from multiple ontologies. Ontology alignment is especially relevant in the semantic web domain as RDF documents and OWL ontologies are quite heterogeneous, yet often describe related concepts. The end goal for ontology matching is to allow the knowledge sets to interoperate. To achieve this goal, it is necessary for queries to return results that include knowledge, and inferred knowledge, from multiple datasets and terminologies, using the alignment information. Furthermore, ontology alignment is not an exact science, and concept matchings often involve uncertainty. The goal of this paper is to provide a semantic web repository that supports applying alignments to the dataset and reasoning with alignments. Our goal is to provide high performance queries that return resul

...

[['https://w3id.org/scholarlydata/inproceedings/iswc2002/proceedings/paper-35', '1256', 'The Usable Ontology: An Environment for Building and Assessing a Domain Ontology', ' ', '', '', '0.3252728217238334'], ['https://w3id.org/scholarlydata/inproceedings/swws2001/position-proceedings/paper-18', '1103', 'Semantics for Scientific Data: Smart Dictionaries as Ontologies', ' ', '', '', '0.32070159224041694'], ['https://w3id.org/scholarlydata/inproceedings/iswc2016/paper/research/research-68', '5378', 'Updating DL-Lite Ontologies through First-Order Queries', ' ', '', '', '0.3158665323011876']] indice 2


  ------------------------------------------------------------ Índex 2
[['https://w3id.org/scholarlydata/inproceedings/swws2001/proceedings/paper-07', '1137', 'Brokerage of Intellectual Property Rights in the Semantic Web', ' ', '', '', '0.3437439157758214'], ['https://w3id.org/scholarlydata/inproceedings/eswc-2017-workshops-ldq-2', '5170', 'Towards Ontology Quality Assessment', 'The succes

[['https://w3id.org/scholarlydata/inproceedings/iswc2013/doctoral-consortium-proceedings/paper-02', '2535', 'Semantic Interpretation of Mobile Phone Records Exploiting Background Knowledge', ' ', '', '', '0.30863369178224426'], ['https://w3id.org/scholarlydata/inproceedings/iswc2005/poster-demo-proceedings/paper-70', '1593', "Toward Task Ontology-based Modeling for Mobile Phone Users' Activity", ' ', '', '', '0.3044584990516327'], ['https://w3id.org/scholarlydata/inproceedings/eswc2014/paper/demo/114', '606', 'Modelling OWL ontologies with Graffoo', ' ', 'https://w3id.org/scholarlydata/person/aldo-gangemi,https://w3id.org/scholarlydata/person/david-shotton,https://w3id.org/scholarlydata/person/fabio-vitali,https://w3id.org/scholarlydata/person/silvio-peroni,https://w3id.org/scholarlydata/person/riccardo-falco', '', '0.30385566788564355']] indice 3


  ------------------------------------------------------------ Índex 3
[] indice 3


  ---------------------------------------------------

[] indice 3


  ------------------------------------------------------------ Índex 3
[['https://w3id.org/scholarlydata/inproceedings/iswc2005/poster-demo-proceedings/paper-71', '1594', 'Towards Browsing Distant Metadata with Semantic Signature', ' ', '', '', '0.3846858878089152'], ['https://w3id.org/scholarlydata/inproceedings/www2008/paper/373', '3757', 'Diverse and Representative Image Search Results for Landmarks', "Can we automatically generate representative and diverse views of the world's landmarks from community-contributed collections on the web? Community-contributed collections of media on the web are a becoming a vast, rich resource for image and video on a long-tailed array of topics.  We use a combination of context- and content-based tools to generate representative sets of images for location-driven features and landmarks, a common search task. To do that, we using location and other metadata, as well as tags associated with images, and the images' visual features. We p

[['https://w3id.org/scholarlydata/inproceedings/lrec2008/papers/547', '3031', 'Building a Bio-Event Annotated Corpus for the Acquisition of Semantic Frames from Biomedical Corpora', 'This paper reports on the design and construction of a bio-event annotated corpus which was developed with a specific view to the acquisition of semantic frames from biomedical corpora. We describe the adopted annotation scheme and the annotation process, which is supported by a dedicated annotation tool. The annotated corpus contains 677 abstracts of biomedical research articles.', 'https://w3id.org/scholarlydata/person/paul-thompson,https://w3id.org/scholarlydata/person/philip-cotter,https://w3id.org/scholarlydata/person/giulia-venturi,https://w3id.org/scholarlydata/person/john-mcnaught,https://w3id.org/scholarlydata/person/simonetta-montemagni,https://w3id.org/scholarlydata/person/sophia-ananiadou,https://w3id.org/scholarlydata/person/andrea-trabucco', 'Semantics,Corpus (creation, annotation, etc.),Text

[['https://w3id.org/scholarlydata/inproceedings/iswc2013/data.semanticweb.org/workshop/ssn/2013/proceedings/paper-05', '2503', 'An Ontology Framework for Water Quality Management', ' ', '', '', '0.33912024937421703'], ['https://w3id.org/scholarlydata/inproceedings/iswc2013/proceedings-1/paper-06', '2627', 'Empirical Study of Logic-Based Modules: Cheap Is Cheerful', 'For ontology reuse and integration, a number of approaches have been devised that aim at identifying modules, i.e., suitably small sets of “relevant” axioms from ontologies. Here we consider three logically sound notions of modules: MEX modules, only applicable to inexpressive ontologies; modules based on semantic locality, a sound approximation of the ﬁrst; and modules based on syntactic locality, a sound approximation of the second (and thus the ﬁrst), widely used since these modules can be extracted from OWL DL ontologies in time polynomial in the size of the ontology. In this paper we investigate the quality of both app

# # Validacion Automatica

In [45]:
vaDF = vmDF.values.tolist()
paperVA = pd.DataFrame()
conta = 0
vaVacia = ''
for i in vaDF:
    if (conta == 3):
        conta = 0
        conta = conta + 1
    indice = i[1]
    uriBas = i[4]
    abstractBas = i[0]
    uriSim = i[16]
    abstractSim = i[7]
    scoreGraphDB = i[14]
    rankGraphDB = i[5]
    valorSpacy = scoreSPACY(abstractBas, abstractSim)
    paperVA =  paperVA.append({'index':indice,'uri paper bas':uriBas,'abstract paper bas':abstractBas,'uri paper sim':uriSim,'abstract paper sim':abstractSim,'Score GraphDB':scoreGraphDB,'Rank GraphDB':rankGraphDB,'Score Spacy':valorSpacy,'Rank Spacy':conta,'error':vaVacia}, ignore_index=True)
    abstractSim = ''
    valorSpacy = 0
    conta = 0
    #print(uriBas,"**",abstractBas,"**",uriSim,"**",abstractSim,"**",scoreGraphDB,"**",rankGraphDB)
paperVA.to_excel('validacion_automatica.xlsx')

Our purpose is to provide end-users a means to query ontology based knowledge bases using natural language queries and thus hide the complexity of formulating a query expressed in a graph query language such as SPARQL. The main originality of our approach lies in the use of query patterns. Our contribution is materialized in a system named SWIP, standing for Semantic Web Interface Using Patterns. The demo will present use cases of this system. 1111111
  2222222
Our purpose is to provide end-users a means to query ontology based knowledge bases using natural language queries and thus hide the complexity of formulating a query expressed in a graph query language such as SPARQL. The main originality of our approach lies in the use of query patterns. Our contribution is materialized in a system named SWIP, standing for Semantic Web Interface Using Patterns. The demo will present use cases of this system. 1111111
  2222222
Our purpose is to provide end-users a means to query ontology based 

The primary challenge of machine perception is to define efficient computational methods to derive high-level knowledge from low-level sensor observation data. Emerging solutions are using ontologies for expressive representation of concepts in the domain of sensing and perception, which enable advanced integration and interpretation of heterogeneous sensor data. The computational complexity of OWL, however, seriously limits its applicability and use within resource-constrained environments, such as mobile devices. To overcome this issue, we employ OWL to formally define the inference tasks needed for machine perception - explanation and discrimination - and then provide efficient algorithms for these tasks, using bit-vector encodings and operations. The applicability of our approach to machine perception is evaluated on a smart-phone mobile device, demonstrating dramatic improvements in both efficiency and scale. 1111111
  2222222
The primary challenge of machine perception is to defi

Existing opinion retrieval techniques do not provide context-dependent relevant results. Most of the approaches used by state-of-the-art techniques are based on frequency of query terms, such that all documents containing query terms are retrieved, regardless of contextual relevance to the intent of the human seeking the opinion. However, in a particular opinionated document, words could occur in different contexts, yet meet the frequency attached to a certain opinion threshold, thus explicitly creating a bias in overall opinion retrieved. In this paper we propose a sentence-level contextual model for opinion retrieval using grammatical tree derivations and approval voting mechanism. Model evaluation performed between our contextual model, BM25, and language model shows that the model can be effective for contextual opinion retrieval such as faceted opinion retrieval. 1111111
  2222222
Existing opinion retrieval techniques do not provide context-dependent relevant results. Most of the 

This paper proposes a novel algorithm to annotate web images by automatically aligning the images with their most relevant auxiliary text terms. First, the DOM-based web page segmentation is performed to extract images and their most relevant auxiliary text blocks. Second, automatic image clustering is used to partition the web images into a set of groups according to their visual similarity contexts, which significantly reduces the uncertainty on the relatedness between the images and their auxiliary terms. The semantics of the visually-similar images in the same cluster are then described by the same ranked list of terms which frequently co-occur in their text blocks. Finally, a relevance re-ranking process is performed over a term correlation network to further refine the ranked term list. Our experiments on a large-scale database of web pages have provided very positive results. 1111111
Comparing and contrasting is an important strategy people adopt to understand new situations and

The huge amount of the available information in the Web creates the need of effective information extraction systems that are able to produce metadata that satisfy user s information needs. The development of such systems, in the majority of cases, depends on the availability of an appropriately annotated corpus in order to learn extraction models. The production of such corpora can be significantly facilitated by annotation tools that are able to annotate, according to a defined ontology, not only named entities but most importantly relations between them. This paper describes the BOEMIE ontology-based annotation tool which is able to locate blocks of text that correspond to specific types of named entities, fill tables corresponding to ontology concepts with those named entities and link the filled tables based on relations defined in the domain ontology. Additionally, it can perform annotation of blocks of text that refer to the same topic. The tool has a user-friendly interface, su

...

The huge amount of the available information in the Web creates the need of effective information extraction systems that are able to produce metadata that satisfy user s information needs. The development of such systems, in the majority of cases, depends on the availability of an appropriately annotated corpus in order to learn extraction models. The production of such corpora can be significantly facilitated by annotation tools that are able to annotate, according to a defined ontology, not only named entities but most importantly relations between them. This paper describes the BOEMIE ontology-based annotation tool which is able to locate blocks of text that correspond to specific types of named entities, fill tables corresponding to ontology concepts with those named entities and link the filled tables based on relations defined in the domain ontology. Additionally, it can perform annotation of blocks of text that refer to the same topic. The tool has a user-friendly interface, su

Opinionated social media like product reviews are now widely used by individuals and organizations for their decision making. However, due to the reason of profit or fame, people try to game the system by opinion spamming (e.g., writing fake reviews) to promote or demote some target products. For reviews to reflect genuine user experiences and opinions, such spam reviews should be detected. Prior works on opinion spam focused on detecting fake reviews and individual spammers. However, a spammer group (a group of reviewers who work collaboratively to write fake reviews) is even more damaging and can take total control of the sentiment on the target product due to its size. This paper studies spam detection in the collaborative setting, i.e., to discover review spammer groups. The proposed method first uses a frequent itemset mining method to find a set of candidate groups. It then uses several behavioral models derived from the collusion phenomenon among spammers and relation models bas

It is a challenging task to match similar or related terms/expressions in NLP and Text Mining applications. Two typical areas in need for such work are terminology and ontology constructions, where terms and concepts are extracted and organized into certain structures with various semantic relations. In the EU BOOTSTrep Project we test various techniques for matching terms that can assist human domain experts in building and enriching ontologies. This paper reports on a work in which we evaluated a text comparing and clustering tool for this task. Particularly, we explore the feasibility of matching related terms with their definitions. Ontology terms, such as Gene Ontology terms, are often assigned with detailed definitions, which provide a fundamental information source for detecting relations between terms. Here we focus on the exploitation of term definitions for the term matching task. Our experiment shows that the tool is capable of grouping many related terms using their definit

Most of today's web content is designed for human consumption, which makes it difficult for software tools to access them readily. Even web content that is automatically generated from back-end databases is usually presented without the original structural information. In this paper, we present an automated information extraction algorithm that can extract the relevant attribute-value pairs from product descriptions across different sites. A notion, called structural-semantic entropy, is used to locate the data of interest on web pages, which measures the density of occurrence of relevant information on the DOM tree representation of web pages. Our approach is less labor-intensive and insensitive to changes in web-page format. Experimental results on a large number of real-life web page collections are encouraging and confirm the feasibility of the approach, which has been successfully applied to detect false drug advertisements on the Web due to its capacity in associating the attribu

In order to reduce the cost of building domain ontologies manually, in this paper, we integrate a domain ontology development environment: DODDLE-OWL and an ontology search engine: Swoogle. We propose a method and a tool for domain ontology construction reusing texts and existing ontologies for a target domain extracted by Swoogle. In the evaluation, we applied the method to a particular field of law and evaluated the acquired ontologies. 1111111
  2222222
The use of natural language identiﬁers as reference for ontology elements—in addition to the URIs required by the Semantic Web standards—is of utmost importance based on their predominance in the human everyday life, i.e. speech or print media. Depending on the context, different names can be chosen for one and the same element, and the same element can be referenced by different names. Here homonymy and synonymy are the main cause of the appearance of ambiguity in perceiving which concrete unique ontology element ought to be referen

In [ ]:
resumen1 = 'The recent emergence of mashup tools has refueled research on end user development, i.e., on enabling end users without programming skills to compose own applications. Yet, similar to what happened with analogous promises in web service composition and business process management, research has mostly focused on technology and, as a consequence, has failed its objective. Plain technology (e.g., SOAP/WSDL web services) or simple modeling languages (e.g., Yahoo! Pipes) dont convey enough meaning to non-programmers.We propose a domain-specific approach to mashups that speaks the language of the user, i.e., that is aware of the terminology, concepts, rules, and conventions (the domain) the user is comfortable with. We show what developing a domain-specific mashup tool means, which role the mashup meta-model and the domain model play and how these can be merged into a domain-specific mashup meta-model.We apply the approach implementing a mashup tool for the research evaluation domain. Our user study confirms that domain-specific mashup tools indeed lower the entry barrier to mashup development.'
resumen2 = 'In this demonstration, we present ResEval Mash, a mashup platform for research evaluation, i.e., for the assessment of the productivity or quality of researchers, teams, institutions, journals, and the like - a topic most of us are acquainted with. The platform is specically tailored to the need of sourcing data about scientic publications and researchers from the Web, aggregating them, computing metrics (also complex and ad-hoc ones), and visualizing them. ResEval Mash is a hosted mashup platform with a client side editor and runtime engine, both running inside a common web browser. It supports the processing of also large amounts of data, a feature that is achieved via the sensible distribution of the respective computation steps over client and server. Our preliminary user study shows that ResEval Mash indeed has the power to enable domain experts to develop own mashups (research evaluation metrics); other mashup platforms rather support skilled developers. The reason for this success is ResEval Mashs domain-specicity.'
valor = scoreSPACY(resumen1, resumen2)
numero = valor[0][0]
print(valor)
print(numero)